In [2]:
import yaml
import json
from openai import OpenAI
import sqlite3
import pandas as pd

In [3]:
with open('api_keys.yaml', 'r') as f:
    api_keys = yaml.safe_load(f)

OPENAI_API_KEY = api_keys['openai-key']
client = OpenAI(api_key=OPENAI_API_KEY)

## Define tools and functions
Need functions to determine what tables to call. Predefined table structure

In [4]:
db_name = "lldm.db"
try: 
    db = sqlite3.connect(db_name) 
    print('Connected to DB')
except: 
    print("DB connection failed")

Connected to DB


In [5]:
# run query and return pandas dataframe
def run_query(db, query):
    try:
        df = pd.read_sql_query(query, db)
        return df
    # not a select statement
    except TypeError:
        cursor = db.cursor()
        cursor.execute(query)
        return    

In [6]:
sql_query = """
SELECT name FROM sqlite_master  
WHERE type='table';
"""
cursor = db.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('CHARACTER_SHEET',), ('INVENTORY',), ('SETTINGS',), ('NPCS',), ('TREASURES',), ('MONSTERS',), ('PLOT',), ('LOGS',), ('CAMPAIGN',), ('CHARACTER_INVENTORY',)]


In [7]:
query = '''
SELECT * FROM INVENTORY;
'''
run_query(db,query)

,Item_ID,Category,Item,Description
0,0,Weapon,Longsword,Elara's main weapon for combat.
1,1,Weapon,Dagger,Versatile tool for close combat and utility pu...
2,2,Adventuring Gear,Backpack,To carry essential items.
3,3,Adventuring Gear,Rope (50 feet),Useful for climbing and securing objects.
4,4,Adventuring Gear,Rations (5 days),Enough food for the journey.
5,5,Adventuring Gear,Water skin,To carry water.
6,6,Adventuring Gear,Flint and Steel,For starting fires.
7,7,Adventuring Gear,Healing Potions (2),For quick recovery in emergencies.
8,8,Armor,Chain Mail,Provides solid protection while allowing mobil...
9,9,Armor,Shield,Additional defense against attacks.


In [8]:
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity


### Picking up item
For now, this is the pipeline:
- Item must come from INVENTORY table in db
- Assume that the item will always be written exactly as it appears in the table
- 

In [9]:
# check if item queried is in INVENTORY table
def validate_item(item_name):
    query = f'''
    SELECT * FROM INVENTORY WHERE UPPER(Item) LIKE "{item_name.upper()}%" 
    '''
    tmp = run_query(db, query)
    if not tmp.empty:
        return tmp['Item_ID'].iloc[0]
    return None

# update table if item validated, otherwise error message
# for now, use temporary campaign and character id
def get_obtained_item(item_name, quantity, campaign_id=0, character_id=0):
    item_id = validate_item(item_name)
    print(item_id, quantity)
    if item_id is not None:
        # TODO: error handling
        cursor = db.cursor()
        query = f'''
        UPDATE CHARACTER_INVENTORY SET Quantity = Quantity + {quantity}
        WHERE Item_ID = {item_id} AND Campaign_ID = {campaign_id} AND Character_ID = {character_id}
        '''
        cursor.execute(query)
        if cursor.rowcount == 0:
            query = f'''
            INSERT INTO CHARACTER_INVENTORY (Campaign_ID, Character_ID, Item_ID, Quantity) VALUES ({campaign_id}, {character_id}, {item_id}, {quantity})
            '''
            cursor.execute(query)
        return json.dumps({'message':'The item(s) were successfully obtained. Please continue the story.'})
    else:
        return json.dumps({'message':'Item does not exist. Please prompt user to specify further or provide another action.'})


def run_conversation_obtain_item(query):
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": query}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_obtained_item",
                "description": "Extract the item that the user has obtained in some manner (such as picked up, purchased, etc.)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "item_name": {
                            "type": "string",
                            "description": "The name of the obtained item.",
                        },
                        "quantity": {
                            "type": "integer",
                            "description": "The number of said items obtained. If a number is not specified, try and infer based on the surrounding context."
                        }
                    },
                    "required": ["item_name", "quantity"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        #tool_choice="auto",  # auto is default, but we'll be explicit
        tool_choice = {"type":'function','function':{'name':'get_obtained_item'}}
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_obtained_item": get_obtained_item,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                item_name=function_args.get("item_name"),
                quantity=function_args.get("quantity"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response


In [10]:
print(run_conversation_obtain_item("I pick up a longsword from the bodies of the dead goblins."))

0 1
ChatCompletion(id='chatcmpl-9gi1JMFNlh1IYyp6tTtDKGUNhPs4d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="As you carefully extricate the longsword from the dead goblin's grip, you take a moment to examine it. The blade is slightly worn but still sharp, with a faint glow that suggests it might be enchanted. The hilt is wrapped in leather, providing a sturdy grip.\n\nWhat would you like to do next? You can explore the area, check the other goblins for additional items, or perhaps continue on your journey. The choice is yours.", role='assistant', function_call=None, tool_calls=None))], created=1719966017, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_d576307f90', usage=CompletionUsage(completion_tokens=92, prompt_tokens=75, total_tokens=167))


In [11]:
# Check to see if tables were updated
# TODO: WHY IS THIS UPDATED TWICE??
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity
0,0,0,0,1.0


In [12]:
query = '''
DELETE FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

### Discarding item
For now, this is the pipeline:
- Item must come from INVENTORY table in db AND must exist in CHARACTER_INVENTORY
- Assume that the item will always be written exactly as it appears in the table

In [13]:
# defining custom exceptions to be used with error handling
class ItemNotFoundException(Exception):
    pass

class ItemNotPossessedException(Exception):
    pass

In [14]:
# check if item queried is in INVENTORY table and character has more than discard amount
# might need a third error condition if item exists, user has item, but has less than discard amount
def validate_item_discard(item_name, quantity, campaign_id, character_id):
    query = f'''
    SELECT * FROM INVENTORY WHERE UPPER(Item) LIKE "{item_name.upper()}%" 
    '''
    tmp = run_query(db, query)
    if not tmp.empty: # item exists in world
        item_id = tmp['Item_ID'].iloc[0]
        # validate if character has item
        query = f'''
        SELECT * FROM CHARACTER_INVENTORY 
        WHERE Campaign_ID = {campaign_id} AND Character_ID = {character_id} AND Item_ID = {item_id} AND Quantity >= {quantity}
        '''
        # there should theoretically only be one row of the item for each character with the quantity as different values
        # need to validate and put checks in place
        tmp = run_query(db, query)
        if not tmp.empty:
            return item_id
        raise ItemNotPossessedException("Character does not have the item in their inventory.")
    raise ItemNotFoundException("Item does not exist in this campaign.")

# update table if item validated, otherwise error message
# for now, use temporary campaign and character id
# first update with item subtraction, then remove all entries with <=0 values
def get_discarded_item(item_name, quantity, campaign_id=0, character_id=0):
    try:
        item_id = validate_item(item_name)
        print(item_id, quantity)
        cursor = db.cursor()
        query = f'''
        UPDATE CHARACTER_INVENTORY SET Quantity = Quantity - {quantity}
        WHERE Item_ID = {item_id} AND Campaign_ID = {campaign_id} AND Character_ID = {character_id};
        '''
        cursor.execute(query)

        # housekeeping query, remove rows with invalid quantites (<= 0 items)
        query = 'DELETE FROM CHARACTER_INVENTORY WHERE Quantity <= 0'
        cursor.execute(query)
        return json.dumps({'message':'The item(s) were successfully discarded. Please continue the story.'})
    except ItemNotPossessedException as e:
        return json.dumps({'message':"Item is not in character's possession. Please prompt user to specify further or provide another action."})
    except ItemNotFoundException as e:
        return json.dumps({'message':"Item does not exist. Please prompt user to specify further or provide another action."})


def run_conversation_discard_item(query):
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": query}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_discarded_item",
                "description": "Extract the item that the user has discarded in some manner (such as thrown away, consumed, broken, etc.)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "item_name": {
                            "type": "string",
                            "description": "The name of the discarded item.",
                        },
                        "quantity": {
                            "type": "integer",
                            "description": "The number of said items discarded. If a number is not specified, try and infer based on the surrounding context."
                        }
                    },
                    "required": ["item_name", "quantity"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
        # tool_choice = {"type":'function','function':{'name':'get_discarded_item'}}
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_discarded_item": get_discarded_item,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(function_args.get("item_name"), function_args.get("quantity"))
            function_response = function_to_call(
                item_name=function_args.get("item_name"),
                quantity=function_args.get("quantity"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response


In [15]:
print(run_conversation_discard_item("I throw my longsword off the cliff overlooking the sea."))

longsword 1
0 1
ChatCompletion(id='chatcmpl-9gi1M3DmoU6eM2kj8NATaTKsQt7di', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your longsword spins in the air, catching the light of the setting sun as it arcs downward before disappearing into the churning waves far below. The sound of the sword meeting the water is swallowed by the roar of the sea and the howl of the wind. With the weight of the weapon now lifted from your shoulders, what do you plan to do next?', role='assistant', function_call=None, tool_calls=None))], created=1719966020, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_ce0793330f', usage=CompletionUsage(completion_tokens=72, prompt_tokens=72, total_tokens=144))


In [16]:
# Check to see if tables were updated
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity


### Combine the two functions

In [17]:
def run_conversation_item(query):
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": query}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_obtained_item",
                "description": "Extract the item that the user has obtained in some manner (such as picked up, purchased, etc.)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "item_name": {
                            "type": "string",
                            "description": "The name of the obtained item.",
                        },
                        "quantity": {
                            "type": "integer",
                            "description": "The number of said items obtained. If a number is not specified, try and infer based on the surrounding context."
                        }
                    },
                    "required": ["item_name", "quantity"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_discarded_item",
                "description": "Extract the item that the user has discarded in some manner (such as thrown away, consumed, broken, etc.)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "item_name": {
                            "type": "string",
                            "description": "The name of the discarded item.",
                        },
                        "quantity": {
                            "type": "integer",
                            "description": "The number of said items discarded. If a number is not specified, try and infer based on the surrounding context."
                        }
                    },
                    "required": ["item_name", "quantity"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
        # tool_choice = {"type":'function','function':{'name':'get_discarded_item'}}
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_obtained_item": get_obtained_item,
            "get_discarded_item": get_discarded_item,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(function_args.get("item_name"), function_args.get("quantity"))
            function_response = function_to_call(
                item_name=function_args.get("item_name"),
                quantity=function_args.get("quantity"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response

In [22]:
# base state of inventory
query = '''
SELECT * FROM INVENTORY;
'''
run_query(db,query)

,Item_ID,Category,Item,Description
0,0,Weapon,Longsword,Elara's main weapon for combat.
1,1,Weapon,Dagger,Versatile tool for close combat and utility pu...
2,2,Adventuring Gear,Backpack,To carry essential items.
3,3,Adventuring Gear,Rope (50 feet),Useful for climbing and securing objects.
4,4,Adventuring Gear,Rations (5 days),Enough food for the journey.
5,5,Adventuring Gear,Water skin,To carry water.
6,6,Adventuring Gear,Flint and Steel,For starting fires.
7,7,Adventuring Gear,Healing Potions (2),For quick recovery in emergencies.
8,8,Armor,Chain Mail,Provides solid protection while allowing mobil...
9,9,Armor,Shield,Additional defense against attacks.


In [19]:
# base state of inventory
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity


In [25]:
print(run_conversation_item("I pick up 5 backpacks from the bodies of the dead goblins."))

backpack 5
2 5
ChatCompletion(id='chatcmpl-9gi7XC1j2FQoQy5G0LGawwSx5ikuZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="You gather the five backpacks from the fallen goblins, examining each one for anything of value or significance. \n\nThe first backpack contains several crude knives, a handful of coins, and some moldy bread. The second one has a small, tattered map which seems to show a part of the forest, perhaps marking some hidden location. The third backpack holds a vial of green liquid, presumably some kind of potion, and a bundle of arrows. The fourth one is filled with various bits of junk and trinkets, including a rusty key. The last backpack, surprisingly, contains a rolled parchment sealed with an unfamiliar emblem.\n\nAs you sift through these items, you can't help but feel a mix of curiosity and caution. What will you do next?", role='assistant', function_call=None, tool_calls=None))], created=1719966403, model='gpt-

In [26]:
# Check to see if tables were updated
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity
0,0,0,0,1.0
1,0,0,1,1.0
2,0,0,2,5.0


In [27]:
print(run_conversation_item("I throw my longsword off the cliff overlooking the sea."))

longsword 1
0 1
ChatCompletion(id='chatcmpl-9gi7s266KWLrWlIpXx6PAaUtNXtDG', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The longsword twirls end over end as it glints in the sunlight, hurtling down towards the sea below. It breaks the surface with a sharp splash and is quickly swallowed by the waves, disappearing from sight.\n\nAs you stand on the cliff's edge, the salty breeze whips at your hair and clothes. The sound of the crashing waves fills the air, mingling with the distant cries of seabirds. There is a certain finality in the act, a sense of release.\n\nWhat do you do next?", role='assistant', function_call=None, tool_calls=None))], created=1719966424, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_d576307f90', usage=CompletionUsage(completion_tokens=105, prompt_tokens=72, total_tokens=177))


In [28]:
# Check to see if tables were updated
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity
0,0,0,1,1.0
1,0,0,2,5.0


In [29]:
print(run_conversation_item("I throw my longsword off the cliff overlooking the sea."))

longsword 1
0 1
ChatCompletion(id='chatcmpl-9giD1Q7DsMFdqOIiqlgbDPLq3LKgW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The longsword arcs gracefully through the air, its blade catching the last rays of the setting sun before it plunges into the churning waves below. As the ocean swallows the weapon, a sense of finality settles over you. What will you do next?', role='assistant', function_call=None, tool_calls=None))], created=1719966743, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_d576307f90', usage=CompletionUsage(completion_tokens=53, prompt_tokens=72, total_tokens=125))


In [30]:
# Check to see if tables were updated
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity
0,0,0,1,1.0
1,0,0,2,5.0
